In [1]:
import time

In [2]:
from juggling_dlx_milp import *

/home/josue/Documents/git/Jugglyng1/Expérimentations/Juggling DLX/DLX/dlxm.py:69: RuntimeWarning: class "DLX_M::AbstrItem" has no virtual destructor
  class ConcItem(_AbstrItem):


In [3]:
configs = HandsConfigurations({"do", "ré", "mi"}, 2)
configs

[(('mi', 'ré'), ('do',)),
 (('ré', 'mi'), ('do',)),
 (('mi', 'do'), ('ré',)),
 (('do', 'mi'), ('ré',)),
 (('ré', 'do'), ('mi',)),
 (('do', 'ré'), ('mi',)),
 (('mi',), ('ré', 'do')),
 (('mi',), ('do', 'ré')),
 (('ré',), ('mi', 'do')),
 (('ré',), ('do', 'mi')),
 (('do',), ('mi', 'ré')),
 (('do',), ('ré', 'mi')),
 ((), ('mi', 'ré', 'do')),
 ((), ('mi', 'do', 'ré')),
 ((), ('ré', 'mi', 'do')),
 ((), ('ré', 'do', 'mi')),
 ((), ('do', 'mi', 'ré')),
 ((), ('do', 'ré', 'mi')),
 (('mi', 'ré', 'do'), ()),
 (('mi', 'do', 'ré'), ()),
 (('ré', 'mi', 'do'), ()),
 (('ré', 'do', 'mi'), ()),
 (('do', 'mi', 'ré'), ()),
 (('do', 'ré', 'mi'), ())]

### Musique à jouer

In [4]:
music = [( 1, "do"), ( 2, "do"), ( 3, "do"), 
         ( 4, "re"), ( 5, "mi"), ( 7, "re"), 
         ( 9, "do"), (10, "mi"), (11, "re"),
         (12, "re"), (13, "do")]

In [5]:
music = [(1, "re5"), (2, "do5"), (3, "re5"), (4, "do5"),
         (5, "re5"), (7, "do5"), (9, "mi5"),
         (11, "re5"), (12, "fa5"), (13, "la5"), (14, "re6"), (15, "fa6")]

In [6]:
music = [( 1, "do"), ( 2, "do")]

### Contraintes

In [7]:
nb_hands = 2
max_height = 5
max_weight = 2
forbidden_multiplex = [(1, 2), (1, 3), (1, 4), (2, )]
multiple_throws = True

### Algorithme

In [8]:
balls, throws = music_to_throws(music)

In [9]:
ec_instance = throws_to_extended_exact_cover(balls, throws, nb_hands, max_height, max_weight,
                                             forbidden_multiplex, [], multiple_throws)

2


In [10]:
dlx = dlx_solver_instance(ec_instance)

In [11]:
# for r in range(len(dlx.rows)):
#     print(dlx.row_obj(r))
len(dlx.rows)

20

In [16]:
for r in range(len(dlx.rows)):
    print(dlx.row_obj(r))

[XItem(throw=Throw(ball='do', time=0, max_height=1), hand=0, flying_time=1), LItem(throw=Throw(ball='do', time=0, max_height=1)), UItem(ball='do', time=0, hand=0), UItem(ball='do', time=1, hand=0), (PItem(ball='do', time=0), 1)]
[XItem(throw=Throw(ball='do', time=0, max_height=1), hand=1, flying_time=1), LItem(throw=Throw(ball='do', time=0, max_height=1)), UItem(ball='do', time=0, hand=1), UItem(ball='do', time=1, hand=1), (PItem(ball='do', time=0), 3)]
[XItem(throw=Throw(ball='do', time=1, max_height=1), hand=0, flying_time=1), LItem(throw=Throw(ball='do', time=1, max_height=1)), UItem(ball='do', time=1, hand=0), UItem(ball='do', time=2, hand=0), (PItem(ball='do', time=1), 1)]
[XItem(throw=Throw(ball='do', time=1, max_height=1), hand=1, flying_time=1), LItem(throw=Throw(ball='do', time=1, max_height=1)), UItem(ball='do', time=1, hand=1), UItem(ball='do', time=2, hand=1), (PItem(ball='do', time=1), 3)]
[BItem(ball='do', time=0), (PItem(ball='do', time=0), 1), (PItem(ball='do', time=1),

In [12]:
ec_instance.colors

[(0, 0), (0, 1), (0, 2), (1, 1), (1, 2)]

In [13]:
sol = dlx.get_solution()

In [14]:
rows = []
for i in sol:
    rows.append(dlx.row_obj(i))
solution = ExactCoverSolution(max_time=ec_instance.max_time,
                              nb_hands=ec_instance.nb_hands,
                              balls=ec_instance.balls,
                              rows=rows)

In [15]:
print_juggling(solution)

{} {} : 
{} {} : 


In [8]:
t = time.time()
sols = solve_with_dlx(ec_instance)
print("{} solutions trouvées en {} secondes".format(len(sols), time.time() - t))

0 solutions trouvées en 0.8061761856079102 secondes


In [17]:
print_juggling(sols[0])

{'do5', 'mi5', 're5', 'fa5'}       {}       : re5 -- 1 --> 1
   {'do5', 'mi5', 'fa5'}        {'re5'}     : do5 -- 1 --> 1
       {'mi5', 'fa5'}        {'do5', 're5'} : re5 -- 1 --> 0
   {'mi5', 'fa5', 're5'}        {'do5'}     : do5 -- 1 --> 0
{'do5', 'mi5', 'fa5', 're5'}       {}       : re5 -- 1 --> 1
   {'do5', 'mi5', 'fa5'}        {'re5'}     : 
   {'do5', 'mi5', 'fa5'}        {'re5'}     : do5 -- 1 --> ?
       {'mi5', 'fa5'}           {'re5'}     : 
       {'mi5', 'fa5'}           {'re5'}     : mi5 -- 1 --> ?
          {'fa5'}               {'re5'}     : 
          {'fa5'}               {'re5'}     : re5 -- 1 --> ?
          {'fa5'}                  {}       : fa5 -- 1 --> ?


In [18]:
import modele
import ipywidgets
import pythreejs

# balls = [{"color":"blue", "tone":"do5", "name":"do5"},
#          {"color":"red", "tone":"re5", "name":"re5"},
#          {"color":"green", "tone":"mi5", "name":"mi5"},
#          {"color":"yellow", "tone":"fa5", "name":"fa5"},
#          {"color":"yellow", "tone":"la5", "name":"la5"},
#          {"color":"yellow", "tone":"re6", "name":"re5"},
#          {"color":"yellow", "tone":"fa6", "name":"fa6"}]

balls = [{"color":"blue", "tone":"do", "name":"do"},
         {"color":"red", "tone":"re", "name":"re"},
         {"color":"green", "tone":"mi", "name":"mi"}]

pattern = juggling_sol_to_simulator(sols[0])

sides = [-1, 1]

model = modele.Model(balls, pattern)
slider = ipywidgets.FloatSlider(min=0, max=40, step=0.1)
view = modele.View(model, sides)
slider.observe(lambda change: view.update(change['new'], change['old']), names="value")
ipywidgets.VBox([view.widget,slider])